In [1]:
from libs import embeddings
from libs.graph import KnowledgeGraph

kg = KnowledgeGraph.from_dir("toy")
#E = embeddings.load("toy")

Triples: 100%|██████████| 316114/316114 [00:02<00:00, 123138.73it/s]


In [2]:
ename = "dbr:Anthony_Warde"
eid = kg.ent.to_id(ename)

kg.print_relations(ename)

dbr:Anthony_Warde
	 dbo:birthDate xsd:date
	 rdf:type
		 dbo:Agent
		 dbo:Person
	 dbo:activeYearsStartYear xsd:gYear
	 dbo:activeYearsEndYear xsd:gYear
	 dbo:imdbId <STRING>
	 dbo:birthPlace dbr:Pennsylvania
	 foaf:givenName <LABEL:en>
	 dbo:birthYear xsd:gYear
	 dbo:occupation dbr:Film
	 dbo:deathDate xsd:date
	 dbo:deathPlace
		 dbr:California
		 dbr:Hollywood
	 foaf:surname <LABEL:en>
	 foaf:gender <LABEL:en>
	 dbo:deathYear xsd:gYear


In [3]:
from importlib import reload
import libs.sampling.instance_sampling as samp
reload(samp)

samp.set_graph(kg)

instances, n = samp.sample_any(10)
print(f"Sampled {len(instances)} instances out of {n}")
print(*map(kg.ent.to_name, instances), sep="\n")

Sampled 10 instances out of 47694
dbr:Kolding
dbr:Gmina_Sławno,_Łódź_Voivodeship
dbr:Cadiz,_Kentucky
dbr:Jafarbay-ye_Jonubi_Rural_District
dbr:Colegio_Niño_Jesús_de_Praga
dbr:Lev_Kamenev
dbr:Andilanatoby
dbr:AC_Sparta_Praha_(women)
dbr:Whiting,_Vermont
dbr:The_Bahamas


In [4]:
# import libs.axiom.atomic as atoms
from libs.axiom import Concept, Existential
#reload(atoms)
reload(samp)
#Existential, Concept
samp.set_graph(kg)

Place = Concept(singleton="dbr:California")
a = Existential("dbo:deathPlace", Place)
b = Existential("dbo:deathYear", None)

instances, n = samp.sample_from(a & b, 10)
print(f"Sampled {len(instances)} instances out of {n} for axiom '{a & b}'")
print(*map(kg.ent.to_name, instances), sep="\n")

Sampled 7 instances out of 7 for axiom '∃dbo:deathPlace.{dbr:California}∧∃dbo:deathYear.⊤'
dbr:Ruth_Roland
dbr:A._A._Allen
dbr:Albert_R._Broccoli
dbr:Brad_Renfro
dbr:Anthony_Warde
dbr:Harvey_Lembeck
dbr:Isaac_Newton_Van_Nuys


In [5]:
kg.print_relations("dbr:California")

dbr:California
	 rdf:type
		 dbo:Location
		 dbo:Place
		 dbo:Region
		 dbo:PopulatedPlace
		 dbo:AdministrativeRegion


In [8]:
from collections import Counter

c = Counter()
for h, t in kg.iter_head_tails(kg.rel.to_id("rdf:type"), as_string=True):
    c[t] += 1

In [10]:
help(kg.sample_instances)

Help on method sample_instances in module libs.graph.knowledge_graph:

sample_instances(n, from_type='owl:Thing', except_type=None, type_rel='rdf:type', as_string=False, force_size=True, exclude_ids=None) method of libs.graph.knowledge_graph.KnowledgeGraph instance



In [12]:
classes = {cls for cls, count in c.most_common(n_classes + 1) if cls != "dbo:Place"}
classes

{'dbo:Agent',
 'dbo:Athlete',
 'dbo:Event',
 'dbo:Location',
 'dbo:Organisation',
 'dbo:Person',
 'dbo:PopulatedPlace',
 'dbo:Settlement',
 'dbo:SocietalEvent',
 'dbo:SportsEvent'}

In [16]:
raw_axioms = """
dbo:Athlete dbo:Person
dbo:Person dbo:Agent
dbo:Organisation dbo:Agent
dbo:SocietalEvent dbo:Event
dbo:SportsEvent dbo:Event
dbo:Settlement dbo:PopulatedPlace
dbo:PopulatedPlace dbo:Location
"""

axioms = [tuple(line.split()) for line in raw_axioms.split("\n")[1:-1]]
axioms

from libs.taxonomy import Taxonomy


T = Taxonomy.from_axioms(axioms, add_root="root")
T.print()

               ┌dbo:SocietalEvent
     ┌dbo:Event┤
     │         └dbo:SportsEvent
 root┤
     │         ┌dbo:Organisation
     ├dbo:Agent┤
     │         └dbo:Person┐
     │                    └dbo:Athlete
     └dbo:Location┐
                  └dbo:PopulatedPlace┐
                                     └dbo:Settlement


In [9]:
from sklearn.utils import shuffle
from libs.dataset import Dataset

n_classes = 10
n_entities = 200
classes = {cls for cls, count in c.most_common(n_classes + 1) if cls != "dbo:Place"}

used_indices = set()
indices = []
labels = []
name2cls, cls2name = dict(), dict()

for name in classes:
    cls = len(name2cls)
    name2cls[name] = cls
    cls2name[cls] = name
    
    for instance in kg.sample_instances(n_entities, from_type=name, exclude_ids=used_indices):
        used_indices.add(instance)
        indices.append(instance)
        labels.append(cls)
        
axioms = {
    "dbo:Athlete", "dbo:Person"
}

dbo:Location dbo:Agent dbo:Event dbo:SportsEvent dbo:Place dbo:PopulatedPlace dbo:Person dbo:SocietalEvent dbo:Organisation dbo:Settlement


In [3]:
# 1: lire les mots
with open("dico.txt", "r", encoding="utf8") as fichier:
    dico = []
    for mot in fichier.read().split("\n"):
        # On enlève les mots trop courts
        if len(mot) > 6:
            dico.append(mot)
            
print(len(dico), "mots trouvés")

311330 mots trouvés


In [7]:
# 2: choisir un mot au hasard
from random import choice
from unidecode import unidecode as deaccent

mot = choice(dico)
mot = deaccent(mot)

print("Mot choisi:", mot)

Mot choisi: lipothymies


In [12]:
début = mot[0]
fin = mot[-1]

mot_à_afficher = début + "_" * (len(mot) - 2) + fin

print(*mot_à_afficher)

l _ _ _ _ _ _ _ _ _ s


In [1]:
nb_étapes = 10
étape_actuelle = 0
lettres_utilisées = []

while étape_actuelle < nb_étapes:
    étape = "  (étape " + str(étape_actuelle) + " sur " + str(nb_étapes) + ")"
    print(*mot_à_afficher, étape)
    lettre = input()
    if lettre in lettres_utilisées:
        print("la lettre", lettre, "a déjà été utilisée")
    else:
        nouveau_mot = ""
        for i in range(len(mot)):
            vraie_lettre = mot[i]
            if vraie_lettre == lettre:
                nouveau_mot += lettre
            else:
                nouveau_mot += mot_à_afficher[i]
            
        étape_actuelle += 1

NameError: name 'mot_à_afficher' is not defined

In [6]:
from random import choice
from unidecode import unidecode

# 1: read word list
min_length = 7
with open("dico.txt", "r", encoding="utf8") as f:
    words = [word for word in f.read().split("\n") if len(word) >= min_length]
print(f"Found {len(words)} words")

Found 311330 words


In [10]:
del word

In [ ]:
from contextlib import redirect_stdout
from io import StringIO
import requests

trash = StringIO()


def get_insult():
    template = "You've been hanged, you <adjective> <adjective min=1 max=3 id=adj1> <amount> of <adjective min=1 max=3> <animal> <animal_part>"
    r = requests.get("https://insult.mattbas.org/api/insult", params=dict(template=template))
    if r.status_code == 200:
        return r.text
    else:
        return "You've been hanged! We can't insult you right now because our insult API seems to be broken. Please try being insulted again in a few minutes."
 
def update_current(letter, current, reference):
    new_word = [l_true if normalize(l_true) == letter else l_curr for l_true, l_curr in zip(reference, current)]
    return "".join(new_word)

def normalize(s):
    return unidecode(str(s).lower())

def play_game(word=None, max_step=5, debug=False):
    if word is None:
        word = choice(words)
    
    TRUE_WORD = word
    if not TRUE_WORD.isalpha():
        raise ValueError(f"'{TRUE_WORD}' is not a valid word")
        
    # word = unidecode(TRUE_WORD)
    inner = normalize(TRUE_WORD[1:-1])
    curr_word = TRUE_WORD[0] + "_" * len(inner) + TRUE_WORD[-1]
    
    if debug:
        print(TRUE_WORD, inner, curr_word)

    print("# GAME START #")
    print(*curr_word)

    won = False
    RUNNING = "running"
    WON = "won"
    DNF = "quit"
    LOST = "lost"
    status = RUNNING
    
    step = 0
    used_letters = set()

    while step < max_step:
        letter = input(f"\n({step}/{max_step})")
        letter = normalize(letter)
        if letter in ("quit", "stop"):
            status = DNF
            break
        if letter in used_letters:
            print(f"Letter '{letter}' has already been used")
            continue
        used_letters.add(letter)
        count = inner.count(letter)
        if not count:
            print(f"Wrong. Letter '{letter}' is not in word")
            step += 1
            continue
        print(f"{count} occurences found for letter '{letter}'")
        curr_word = update_current(letter, curr_word, TRUE_WORD)
        if curr_word == TRUE_WORD:
            print(*curr_word)
            print(f"You won!")
            won = True
            status = WON
            break
        else:
            used = "".join(sorted(used_letters))
            print(*curr_word, f"\t used: {used}")
    else:
        status = LOST
        print(get_insult())
        print(f"The word was '{word}'")
    
    result = dict(
        won=won,
        status=status,
        steps=step,
        word=TRUE_WORD,
        letters=used_letters
    )
    return result

def play_games(n_games=float("inf"), max_step=5):
    curr_game = 0
    results = []
    n_wins = 0
    while curr_game < n_games:
        curr_game += 1
        res = play_game(max_step=max_step)
        if res["won"]:
            n_wins += 1
        results.append(res)
        if curr_game < n_games:
            x = input("Keep playing? [y]/n")
            if x == "n":
                break
    print(f"\nYou won {n_wins} games out of {curr_game} ({100*n_wins/curr_game:.1f}%)")
    return results

res = play_games(5)

In [26]:
def get_insult():
    template = "You lost, you <adjective> <adjective min=1 max=3 id=adj1> <amount> of <adjective min=1 max=3> <animal> <animal_part>"
    r = requests.get("https://insult.mattbas.org/api/insult", params=dict(template=template))
    if r.status_code == 200:
        return r.content
    else:
        return "You lost! We can't insult you right now because our insult API seems to be broken. Please try being insulted again in a few minutes."
    
get_insult()

b'You lost, you savage lesser hostile plate of slimy pig slime'

In [28]:
r.text

'You lost, you wicked toxic pile of clumsy lizard slime'

In [7]:
max_step = 8
step = 0
used_letters = set()
while step < max_step:
    letter = input(f"({step}/{max_step})")
    if letter in ("quit", "stop"):
        break
    if letter in used_letters:
        print(f"Letter '{letter}' has already been used")
        continue
    used_letters.add(letter)
    count = inner.count(letter)
    if not count:
        print(f"Wrong. Letter '{letter}' is not in word")
        step += 1
        continue
    print(f"{count} occurences found for letter '{letter}'")
    curr_word = "".join(letter if letter == true_letter else curr_letter for true_letter, curr_letter in zip(word, curr_word))
    if curr_word == word:
        print(*curr_word)
        print(f"You won!")
        n_won += 1
    else:
        used = "".join(sorted(used_letters))
        print(*curr_word, f"\t used: {used}")
else:
    print(f"You've been hanged! The word was '{word}'")
n_games += 1

trempèrent tremperent remperen t________t


In [ ]:
word = choice(words)
inner = word[1:-1]
curr_word = word[0] + "_" * (len(inner)) + word[-1]

print("# GAME START #")
print(*curr_word)

n_games = 0
n_won = 0

max_step = 8
step = 0
used_letters = set()
while step < max_step:
    letter = input(f"({step}/{max_step})")
    if letter in used_letters:
        print(f"Letter '{letter}' has already been used")
        continue
    used_letters.add(letter)
    count = inner.count(letter)
    if not count:
        print(f"Wrong. Letter '{letter}' is not in word")
        step += 1
        continue
    print(f"{count} occurences found for letter '{letter}'")
    curr_word = "".join(letter if letter == true_letter else curr_letter for true_letter, curr_letter in zip(word, curr_word))
    if curr_word == word:
        print(*curr_word)
        print(f"You won!")
        n_won += 1
    else:
        used = "".join(sorted(used_letters))
        print(*curr_word, f"\t used: {used}")
else:
    print(f"You've been hanged! The word was '{word}'")
n_games += 1


In [5]:
word = choice(words)
inner = word[1:-1]
curr_word = word[0] + "_" * (len(inner)) + word[-1]

word, inner, curr_word

('dereglasse', 'ereglass', 'd________e')